## Trying an Automated Regex

### Getting the Initial Dataframe

In [1]:
import pandas as pd
import re

# File path of the log file
file_path = 'OpenSSH_2k.log'

# Read the first 10 lines from the log file
with open(file_path, 'r') as file:
    log_data = [next(file) for _ in range(10)]

# Convert the log data to a single string to send to the LLM
log_data_str = "\n".join(log_data)

# Function to parse log lines
def parse_logs(log_lines):
    log_pattern = re.compile(r'(?P<date>\w+ \d+ \d+:\d+:\d+) (?P<host>\w+) (?P<process>\w+\[\d+\]): (?P<message>.*)')
    parsed_data = []
    for line in log_lines:
        match = log_pattern.match(line)
        if match:
            parsed_data.append(match.groupdict())
    return pd.DataFrame(parsed_data)

# Initial DataFrame creation
df = parse_logs(log_data)
df.head(3)

,date,host,process,message
0,Dec 10 06:55:46,LabSZ,sshd[24200],reverse mapping checking getaddrinfo for ns.ma...
1,Dec 10 06:55:46,LabSZ,sshd[24200],Invalid user webmaster from 173.234.31.186
2,Dec 10 06:55:46,LabSZ,sshd[24200],input_userauth_request: invalid user webmaster...


### Sending to LLM and Getting Suggestions

In [2]:
import requests
import json

# URL of your local LLM
url = "http://localhost:11434/api/generate"

# Function to send data to an LLM and get suggestions
def get_llm_suggestions(log_data_str):
    prompt = f"Analyze the following log data and suggest additional columns and structure to be added:\n\n{log_data_str}"
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "stream": False
    }
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        try:
            response_data = response.json()
            return response_data
        except json.JSONDecodeError:
            print("Error: Unable to decode JSON response from LLM server.")
            return None
    else:
        print(f"Error: LLM server returned status code {response.status_code}")
        return None

# Get suggestions from the LLM
suggestions = get_llm_suggestions(log_data_str)

if suggestions:
    print("**Additional Columns and Structure Suggestions:**")
    print(suggestions.get('response'))

else:
    print("Failed to retrieve suggestions from the LLM.")


**Additional Columns and Structure Suggestions:**
Based on the provided log data, I suggest adding the following columns and structure to improve analysis and visualization:

1. **Timestamp (UTC)**: Include the timestamp in a standard format (e.g., ISO 8601) to facilitate time-series analysis.
2. **Log Level**: Add a column indicating the log level (e.g., INFO, WARNING, ERROR). This will help categorize logs by severity.
3. **Remote IP**: Extract the remote IP address from each log entry and add it as a separate column. This will enable easy filtering and grouping by source IP.
4. **User Agent**: Include information about the user agent (e.g., webmaster) in a dedicated column. This will help identify patterns and trends in attempted login activities.
5. **Authentication Method**: Add a column indicating the authentication method used (e.g., pam_unix). This will enable analysis of specific authentication mechanisms.
6. **Reason for Failure**: Extract and add a column describing the reas

### Sending Suggestions to Get Code

In [3]:
# Assuming suggestions contain the LLM's suggestions as a string
suggestions_response = suggestions.get('response', '')

# Create the prompt for the LLM to generate implementation code
prompt_2 = f"Generate the code to implement the following suggestions on a pandas DataFrame named 'df':\n\n{suggestions_response}"

# Function to send the second prompt to the LLM and get the implementation code
def get_implementation_code(prompt_2):
    payload = {
        "model": "llama3",
        "prompt": prompt_2,
        "stream": False
    }
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        try:
            response_data = response.json()
            return response_data.get('response', '')
        except json.JSONDecodeError:
            print("Error: Unable to decode JSON response from LLM server.")
            return None
    else:
        print(f"Error: LLM server returned status code {response.status_code}")
        return None

# Get the implementation code from the LLM
implementation_code = get_implementation_code(prompt_2)

if implementation_code:
    print("**Implementation Code:**")
    print(implementation_code)
else:
    print("Failed to retrieve implementation code from the LLM.")


**Implementation Code:**
Here's an example code snippet in Python using pandas that implements these suggestions on a DataFrame named 'df':

```Python
import pandas as pd
from datetime import datetime

# Assuming df is your original DataFrame
df = pd.DataFrame({
    'timestamp': ['Dec 10 06:55:46', ...], 
    'log_level': [...], 
    'remote_ip': [...], 
    'user_agent': [...], 
    'authentication_method': [...], 
    'reason_for_failure': [...], 
    'port_number': [...], 
    'connection_status': [...]
})

# Convert timestamp to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Add a column for UTC time in ISO 8601 standard
df['Timestamp (UTC)'] = df['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S%z')

# Create a log level mapping and apply it to the DataFrame
log_level_mapping = {'ERROR': 'ERROR', 'WARNING': 'WARNING', 'INFO': 'INFO'}
df['Log Level'] = df['log_level'].map(log_level_mapping)

# Extract remote IP from each log entry and add it as a separate column
rem

### Applying the Code

In [4]:
# Initialize the log_table and ssh_connection_log DataFrames
log_table = pd.DataFrame(columns=['Timestamp', 'SSH Connection ID', 'Client IP', 'User Attempted', 'Authentication Result', 'Reason Code'])
ssh_connection_log = pd.DataFrame(columns=['SSH Connection ID', 'Start Time', 'End Time', 'Bytes Transferred'])

# Function to convert date to ISO 8601 format
def convert_to_iso_format(date_str):
    from datetime import datetime
    date_obj = datetime.strptime(date_str, '%b %d %H:%M:%S')
    date_obj = date_obj.replace(year=datetime.now().year)  # Assuming current year
    return date_obj.isoformat() + 'Z'

# List to hold the new rows for log_table and ssh_connection_log
log_table_rows = []
ssh_connection_log_rows = []

# Process each row in the original DataFrame df to populate log_table and ssh_connection_log
for index, row in df.iterrows():
    # Extract details from the message
    timestamp = convert_to_iso_format(row['date'])
    ssh_connection_id = int(re.search(r'sshd\[(\d+)\]', row['process']).group(1))
    client_ip_match = re.search(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', row['message'])
    client_ip = client_ip_match.group(0) if client_ip_match else 'Unknown'
    user_attempted_match = re.search(r'user (\w+)', row['message'])
    user_attempted = user_attempted_match.group(1) if user_attempted_match else 'Unknown'
    authentication_result = 1 if 'invalid user' in row['message'] or 'unknown user' in row['message'] else 0
    reason_code = 'invalid user' if 'invalid user' in row['message'] else 'unknown user' if 'unknown user' in row['message'] else 'success'

    # Add to log_table
    log_table_rows.append({
        'Timestamp': timestamp,
        'SSH Connection ID': ssh_connection_id,
        'Client IP': client_ip,
        'User Attempted': user_attempted,
        'Authentication Result': authentication_result,
        'Reason Code': reason_code
    })

    # Example data for ssh_connection_log (using dummy values for Start Time, End Time, and Bytes Transferred)
    # In real implementation, these values should be extracted from the log data if available
    start_time = timestamp
    end_time = timestamp  # Dummy value, should be replaced with actual end time if available
    bytes_transferred = 1024  # Dummy value, should be replaced with actual bytes transferred if available

    # Add to ssh_connection_log
    ssh_connection_log_rows.append({
        'SSH Connection ID': ssh_connection_id,
        'Start Time': start_time,
        'End Time': end_time,
        'Bytes Transferred': bytes_transferred
    })

# Convert lists to DataFrames and concatenate with the original DataFrames
log_table = pd.concat([log_table, pd.DataFrame(log_table_rows)], ignore_index=True)
ssh_connection_log = pd.concat([ssh_connection_log, pd.DataFrame(ssh_connection_log_rows)], ignore_index=True)

# Convert the 'Timestamp' column in log_table to datetime format
log_table['Timestamp'] = pd.to_datetime(log_table['Timestamp'])

# Convert the 'Start Time' and 'End Time' columns in ssh_connection_log to datetime format
ssh_connection_log['Start Time'] = pd.to_datetime(ssh_connection_log['Start Time'])
ssh_connection_log['End Time'] = pd.to_datetime(ssh_connection_log['End Time'])

# Print the resulting DataFrames
print("Log Table:")
print(log_table)
print('/n')
print("\nSSH Connection Log Table:")
ssh_connection_log.head(3)


Log Table:
                  Timestamp SSH Connection ID       Client IP User Attempted  \
0 2024-12-10 06:55:46+00:00             24200  173.234.31.186        Unknown   
1 2024-12-10 06:55:46+00:00             24200  173.234.31.186      webmaster   
2 2024-12-10 06:55:46+00:00             24200         Unknown      webmaster   
3 2024-12-10 06:55:46+00:00             24200         Unknown        unknown   
4 2024-12-10 06:55:46+00:00             24200  173.234.31.186        Unknown   
5 2024-12-10 06:55:48+00:00             24200  173.234.31.186      webmaster   
6 2024-12-10 06:55:48+00:00             24200  173.234.31.186        Unknown   
7 2024-12-10 07:02:47+00:00             24203  212.47.254.145        Unknown   
8 2024-12-10 07:07:38+00:00             24206    52.80.34.196          test9   
9 2024-12-10 07:07:38+00:00             24206         Unknown          test9   

  Authentication Result   Reason Code  
0                     0       success  
1                     0     

,SSH Connection ID,Start Time,End Time,Bytes Transferred
0,24200,2024-12-10 06:55:46+00:00,2024-12-10 06:55:46+00:00,1024
1,24200,2024-12-10 06:55:46+00:00,2024-12-10 06:55:46+00:00,1024
2,24200,2024-12-10 06:55:46+00:00,2024-12-10 06:55:46+00:00,1024
